# DNSmasq

DNSmasq provides a local DNS server, allowing you to resolve private DNS entires, including wildcards, to localhost for a development cluster.   This allows us to closely mimic a production environment for local testing.  For valid URLs owned by our organization, we can also setup valid TLS certificates and perform full L7 routing to our cluster, allowing us to expose many applications running in a local cluster for development and testing.

## Prerequisiites
- [Learning Jupyter](../../getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - Windows... coming soon!
  - [Mac](../../getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](../../getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](../../kubernetes/install_k8s_cluster.ipynb)

## Install DNSmasq 
Since we're on a Mac, we're going to use 'brew' to install it:

<div class="alert alert-block alert-info">
    <b>Note: </b> 
If you do not already have Homebrew for MacOS, install this first using the following link:
https://brew.sh/
</div>

In [ ]:
brew install dnsmasq

In [ ]:
brew list | grep -i dnsmasq

### Create passsowrd file for scripts
In your Jupyter notebook working directory, create a file called "password" and put your root password in this file. 

<div class="alert alert-block alert-info">
    <b>Note: </b> 
    This is to enable Jupyter notebook to execute sudo commands without exposing your root password or storing it in your machines CLI history while creating a variable. It can be deleted afterwards and there is step included in this notebook to do so. 
</div>

You can see your Jupyter notebook working directory by executing the following:

In [ ]:
pwd

## Configure wildcard DNS entry
Next we will add a rule to the configuration for dnsmasq that routes any URL ending in *.k8s.example.com* to 127.0.0.1.  The code below checks first to see if that line already exists and adds it if it does not:

In [ ]:
if grep -Fxq "address=/.k8s.example.com/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
then
    echo "DNSMasq rule for .k8s.example.com top-level domain already exists"
else
    cat password | sudo -S echo "address=/.k8s.example.com/127.0.0.1" >> /opt/homebrew/etc/dnsmasq.conf
    grep "address=/.k8s.example.com/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
fi

From here we will need to use *brew* to restart the dnsmasq service so that the new configuration can be loaded & used:

In [ ]:
cat password | sudo -S brew services restart dnsmasq

<br>

Use `dig` to test that URLs ending in `.k8s.example.com` resolve to 127.0.0.1 on the DNS server running at 127.0.0.1.  The `@127.0.0.1` option tells dig to use the DNS server running on localhost, the `dnsmasq` service we started earlier.  Note that the `ANSWER SECTION` provides the response for `fubar.k8s.example.com` as `127.0.0.1`.

In [1]:
dig @127.0.0.1 test.k8s.example.com


; <<>> DiG 9.10.6 <<>> @127.0.0.1 test.k8s.example.com
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 14900
;; flags: qr aa rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;test.k8s.example.com.		IN	A

;; ANSWER SECTION:
test.k8s.example.com.	0	IN	A	127.0.0.1

;; Query time: 0 msec
;; SERVER: 127.0.0.1#53(127.0.0.1)
;; WHEN: Mon Mar 07 09:54:07 CST 2022
;; MSG SIZE  rcvd: 65



## Setup Mac DNS resolver

Next, we'll want to override the Mac's DNS resolver for all URLs ending in `.k8s.example.com`, sending only those URLs to our dnsmasq service.  The mac uses "/etc/resolver" for this purpose.  This will allow us to use any browser, such as Chrome or Safari, to use `dnsmasq` to resolve URLs ending in `.k8s.example.com`.

If there is a file in `/etc/resolver` with a filename that matches the end of the URL, then the contents of that file will identify the DNS server to use for that URL.  In our case, we create a file named `/etc/resolver/k8s.example.com` that configures handling of all URLs ending in `.k8s.example.com`, a fictitious top-level domain that I created for test purposes.  This easily could be a subdomain of an actual domain owned by the cluster operator (for example, to listen to any URLs ending in `.k8s.example.com`, the full filename would be `/etc/resolver/k8s.example.com`

In [ ]:
cat password | sudo -S mkdir -p /etc/resolver

In [ ]:
if grep -Fxq "nameserver 127.0.0.1" /etc/resolver/k8s.example.com
then
    echo "MacOS DNS resolver for .k8s.example.com top-level domain already defers to localhost"
else
    cat password | sudo -S sh -c 'echo "nameserver 127.0.0.1" >> /etc/resolver/k8s.example.com'
    cat /etc/resolver/k8s.example.com
fi

### Cleanup - remove password File

In [ ]:
rm password

### Finished 

You now have configured DNSmasq.

### Next Steps
- [Install Monitoring](../../monitoring/install_kube-prometheus-stack.ipynb)
